In [1]:
import datetime as dt
import time
from news_clustering import NewsClustering
from news_processor import NewsProcessor
from json_loader import JsonLoader
from stories_visualizer import visualize_trending_stories
from keywords_post_process import KeywordsPostProcessor
import pandas as pd
from ipywidgets import interact
pd.set_option('display.max_colwidth', None)

In [3]:
@interact
def choose_time_span(date_start='20031106', date_finish='20031230'):
    if __name__ == "__main__":
        
        # uncomment the following lines of codes to rerun keywords extraction
        news_articles = NewsProcessor.load_news_csv(date_start, date_finish, file="data/abcnews-date-text.csv")
        display(news_articles[:5])
        
        t0 = time.time()
        news_articles = NewsProcessor.extract_keywords_news_list(news_articles)
        news_articles = [_d for _d in news_articles if _d['kwords']]
        JsonLoader.save_json(filename="data/abcnews-date-text2003 11 12.json", data=news_articles)
        t1 = time.time()
        print("number of articles {}, keywords extraction time {}s".format(len(news_articles), t1-t0))
        
        #KeywordsPostProcessor.post_process_article_keywords()
        news_articles = JsonLoader.load_json("data/abcnews-date-text2003 11 12.json")
        for _d in news_articles:
            _d['pubDate'] = dt.datetime.strptime(str(_d['pubDate']), "%Y%m%d")
        news_clusters = NewsClustering.cluster_news_by_weighted_keywords(news_articles, eps=0.35, max_size=500)
        NewsClustering.get_cluster_id_and_core_sample_news(news_clusters, news_articles)
        not_clustered = [_d for _d in news_articles if _d['cluster_id'] is None]
        if not_clustered:
            print("number of news not clustered {}".format(len(not_clustered)))
            news_articles = [_d for _d in news_articles if _d['cluster_id'] is not None]
        visualize_trending_stories(news_articles)

interactive(children=(Text(value='20031106', description='date_start'), Text(value='20031230', description='da…

In [7]:
import json
files=['data/abcnews-date-text2003 02 05.json','data/abcnews-date-text2003 05 08.json', 'data/abcnews-date-text2003 08 11.json']

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(f1, 'r') as infile:
            result.extend(json.load(infile))

    with open('abcnews_preprocessed.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)